## Preprocessing

In [61]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [62]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df = application_df.drop(["EIN", "NAME"], 1)

/var/folders/vf/0870f1194_sctn_8jfbdfh9m0000gn/T/ipykernel_26811/1213274594.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(["EIN", "NAME"], 1)


In [63]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
application_df.uunique()

AttributeError: 'DataFrame' object has no attribute 'uunique'

In [64]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
apptype_count = application_df.APPLICATION_TYPE.value_counts()
apptype_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [65]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
application_types_to_replace = list(apptype_count[apptype_count < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [66]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
classification_count=application_df.CLASSIFICATION.value_counts()
print(classification_count)
print("-"*50)
print(f"Average: {classification_count.mean()}")
print(f"Median: {classification_count.median()}") 

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64
--------------------------------------------------
Average: 483.0845070422535
Median: 5.0


In [67]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
clfs_to_remove = pd.DataFrame(classification_count.loc[lambda x : x == 1])
clfs_to_remove_list = clfs_to_remove.index.tolist()

In [68]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace = list(classification_count[classification_count < 500 ].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [69]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
application_df_dummies = pd.get_dummies(application_df,
                                        columns = ["APPLICATION_TYPE", 
                                                   "AFFILIATION", 
                                                   "CLASSIFICATION", 
                                                   "USE_CASE", 
                                                   "ORGANIZATION", 
                                                   "INCOME_AMT", 
                                                   "SPECIAL_CONSIDERATIONS"], 
                                        drop_first = True)

In [70]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
y = application_df_dummies.IS_SUCCESSFUL
X = application_df_dummies.drop(columns="IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [71]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [72]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_feature_count = len(X_train_scaled[0])
input_feature_count

hidden_nodes_layer1 = 150
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 3

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="sigmoid", input_dim = input_feature_count))

# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Added a third hidden layer to improve accuracy
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="softmax"))

# Added a fourth hidden layer to improve accuracy
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))
# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 150)               5700      
                                                                 
 dense_11 (Dense)            (None, 50)                7550      
                                                                 
 dense_12 (Dense)            (None, 10)                510       
                                                                 
 dense_13 (Dense)            (None, 10)                110       
                                                                 
 dense_14 (Dense)            (None, 1)                 11        
                                                                 
Total params: 13,881
Trainable params: 13,881
Non-trainable params: 0
_________________________________________________________________


In [73]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [75]:
# Train the model
#  YOUR CODE GOES HERE
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5360 - accuracy: 0.7394
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5361 - accuracy: 0.7397
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5359 - accuracy: 0.7405
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5356 - accuracy: 0.7393
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5360 - accuracy: 0.7399
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5356 - accuracy: 0.7398
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5358 - accuracy: 0.7401
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5356 - accuracy: 0.7399
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5356 - accuracy: 0.7402
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5357 - accura

In [76]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5600 - accuracy: 0.7282 - 460ms/epoch - 2ms/step
Loss: 0.5600461959838867, Accuracy: 0.7281632423400879


In [77]:
len(nn.get_weights())

10

In [78]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
nn.save("AlphabetSoupCharity_Optimization.h5")